In [3]:
import pandas as pd
import numpy as np
import math
from operator import itemgetter
import statistics
from datetime import datetime, timedelta
import time
import warnings


warnings.simplefilter(action='ignore', category=FutureWarning)


reaction_date = '30.06.2023'
week_number = 26
cluster_number = 2

cluster_name = 'cluster_' + str(week_number) + '_' + str(cluster_number) + '.xlsx'
# unique_name = 'unique_' + str(week_number) + '_' + str(cluster_number) + '.xlsx'
sp_name = 'пц_' + str(week_number) + '_'+ str(cluster_number) + '.xlsx'
pp_name = 'зц_' + str(week_number) + '_' + str(cluster_number) + '.xlsx'
bc_name = 'бу_' + str(week_number) + '_' + str(cluster_number) + '.xlsx'
# rp_name = 'кц_' + str(week_number) + '_' + str(cluster_number) + '.xlsx'
final_name ='final_' + str(week_number) + '_' + str(cluster_number) + '.xlsx'
hp_name = 'hp_file_to_sap_' + str(week_number) + '_' + str(cluster_number) + '.xlsx'
zsd_name = 'zsd_prices_' + str(week_number) + '_' + str(cluster_number) + '.xlsx'
# zsd_pf_name = 'zsd_prices_pf_' + str(week_number) + '_' + str(cluster_number) + '.xlsx'
# zsd_special_name = 'zsd_prices_special_' + str(week_number) + '_' + str(cluster_number) + '.xlsx'
new_promo_name = 'promo_prices_new_' + str(week_number) + '_' + str(cluster_number) + '.xlsx'
new_bc_name = 'new_bc_' + str(week_number) + '_' + str(cluster_number) + '.xlsx'
change_promo_name = 'promo_prices_change_' + str(week_number) + '_' + str(cluster_number) + '.xlsx'
# special_promo_name = 'promo_prices_special_' + str(week_number) + '_' + str(cluster_number) + '.xlsx'

start = time.time()


#Доп массив (max_m1_pg[#][]) и номер элементов
#0 Группа закупок
#1 максимальная м1 (%)

#Доп массив (max_m1_sc[#][]) и номер элементов
#0 Подкатегория
#1 максимальная м1 (%)

###################################################################################
#Доп массив (deeper_reaction[#][]) и номер элементов
#0 Город (название)
#1 Конкурент (СЕТЬ)
#2 углубленная реакция (%)
###################################################################################

#Доп массив (unique_g_o[#][])
#0 ТК >>> СО
#1 Товар

#Доп массив (main_info_1[#][])
#0 Кластер (город)
#1 Подкатегория
#2 ГЗ
#3 EPP
#4 ЧТМ
#5 Наименование
#6 Сеть минимального (с адресом)
#7 мин рег цена конкурентов

#Доп массив (selling_prices[#][])
#0 Товар
#1 СО
#2 Группа кластеров (название)
#3 Кластер (город)
#4 ТК
#5 Действует с (дата)
#6 ПЦ с НДС
#7 РА Ленты

#Доп массив (purchasing_prices[1+#][])
#0 Товар
#1 СО
#2 ЗЦ без НДС
#3 Дата с
#4 НДС
#5 Код города
#6 цена города
#7 ЗЦ НДС

#Массив (zsd_prices[#][])
#0 СО
#1 Кластер (код)
#2 Товар
#3 Цена итог
#4 Дата цен с


#Основной массив (main_info[#][]) и номера элементов
#0 +СО (нужен файл с зо)
#1 +ТК
#2 +EPP
#3 +ЧТМ
#4 +ГЗ
#5 +Подкатегория
#6 +Город (кластер)
#7 +САП
#8 +Наименование
#9 ЗЦ с НДС (нужен файл с зц – ???сначала создать файл с уникальными зо и сап, а потом уже попросить файл с зц???)
#10 +ПЦ текущая (нужен файл с пц - ???сначала создать файл с уникальными зо и сап, а потом уже попросить файл с пц???)
#11 М1 текущая
#12 +РА Ленты (тот же файл с пц, что и выше)
#13 +Цена итог (заполняется во время расчета)
#14 +М1 итог (заполняется во время расчета)
#15 +Отклонение итог (заполняется во время расчета)
#16 +Комментарий (заполняется во время расчета)
#17 +Минимальная регулярная цена конкурентов
#18 +Сеть наименьшего конкурента
#19 +Кол-во конкурентов
#20 +Конкурент 1
#21+ +Конкурент Х



#ИМПОРТ: шаг реакции 5%
###################################################################

reaction_step = pd.read_excel("(обязательно).xlsx",
                          sheet_name="5%",
                          skiprows=0,
                          usecols='A:B',
                          dtype=str)

reaction_step = reaction_step.transpose()
reaction_step = reaction_step.values.tolist()

#ИМПОРТ: исключение товаров из пилота
###################################################################

sku_exclude = pd.read_excel("(обязательно).xlsx",
                          sheet_name="исключить из пилота",
                          skiprows=0,
                          usecols='A:B',
                          dtype=str)

sku_exclude = sku_exclude.transpose()
sku_exclude = sku_exclude.values.tolist()

#ИМПОРТ: расценка угля 
###################################################################

coal_price = pd.read_excel("(обязательно).xlsx",
                          sheet_name="уголь",
                          skiprows=0,
                          usecols='A:C',
                          dtype=str)

coal_price = coal_price.transpose()
coal_price = coal_price.values.tolist()

#ИМПОРТ: макс цена (EDPP)
###################################################################

# max_price_edpp = pd.read_excel("(обязательно).xlsx",
#                           sheet_name="Макс цена EDPP",
#                           skiprows=0,
#                           usecols='A:B',
#                           dtype=str)

# max_price_edpp = max_price_edpp.transpose()
# max_price_edpp = max_price_edpp.values.tolist()

#ИМПОРТ: ср м1 (бренд)
###################################################################
avg_m1_br = pd.read_excel("(обязательно).xlsx",
                          sheet_name="ср м1 (бренд)",
                          skiprows=0,
                          usecols='A:C',
                          dtype=str)

avg_m1_br = avg_m1_br.transpose()
avg_m1_br = avg_m1_br.values.tolist()

#ИМПОРТ: ср м1 (ЧТМ)
###################################################################
avg_m1_pl = pd.read_excel("(обязательно).xlsx",
                          sheet_name="ср м1 (ЧТМ)",
                          skiprows=0,
                          usecols='A:C',
                          dtype=str)

avg_m1_pl = avg_m1_pl.transpose()
avg_m1_pl = avg_m1_pl.values.tolist()

#ИМПОРТ: макс м1 (бренд)
##################################################################
max_m1_br = pd.read_excel("(обязательно).xlsx",
                       sheet_name="макс м1 (бренд)",
                       skiprows=0,
                       usecols='A:B',
                       dtype=str)

max_m1_br = max_m1_br.transpose()
max_m1_br = max_m1_br.values.tolist()

number_max_m1_br = len(max_m1_br[0])

#ИМПОРТ: макс м1 (ЧТМ)
##################################################################
max_m1_pl = pd.read_excel("(обязательно).xlsx",
                       sheet_name="макс м1 (ЧТМ)",
                       skiprows=0,
                       usecols='A:B',
                       dtype=str)

max_m1_pl = max_m1_pl.transpose()
max_m1_pl = max_m1_pl.values.tolist()

number_max_m1_pl = len(max_m1_pl[0])



#ИМПОРТ: основной массив
##################################################################
# #файлы для получения уникальных товаров и со
# print("Введите название файла из САП портала (без расширения).")
# main_info_file_name = str(input())
# #"MonExportCluster1664447352147.xlsx"
# #вводить без расширения
# print("Введите дату начала цены (ДД.ММ.ГГГГ).")
# zsd_prices_date = str(input())
# print("Введите номер недели.")
# week_number = int(input())


print("...Подготовка уникальных СО и товаров...")
print (str(time.time() - start))
unique_g_o = pd.read_excel(cluster_name,
                     skiprows=1,
                     usecols='B, O')

unique_g_o = unique_g_o.transpose()
unique_g_o = unique_g_o.values.tolist()
for i in range(len(unique_g_o[0])):
    unique_g_o[0][i] = int(unique_g_o[0][i][6:])

    
tk_so_cl = pd.read_excel("(обязательно).xlsx",
                     sheet_name="тк-со-кластер",
                     skiprows=0,
                     usecols='A:G',
                     dtype=str)

tk_so_cl = tk_so_cl.transpose()
tk_so_cl = tk_so_cl.values.tolist()
# for i in range(len(tk_so_cl[0])):
#     tk_so_cl[0][i] = int(tk_so_cl[0][i])
#     tk_so_cl[2][i] = str(tk_so_cl[2][i])
    








#передаем некоторые значения в основной массив
goods_number = len(unique_g_o[1])
#20 - основная информация, включая 0 столбец
#20 - цены конкурентов
#20 - сети конкурентов
#1 - медиана по ценам конкурентов
#1 - второе минимальное
#1 - номер поставщика
#1 - доля промо
#1 - рейтинг
#1 - новая цена с (акция + 15 дней)
main_info_columns_number = 78
main_info = [0] * (main_info_columns_number)
for i in range(main_info_columns_number):
    main_info[i] = [0] * goods_number

#заменяем тк на со и передаем в основной массив тк и др.
for i in range(goods_number):
    main_info[1][i] = unique_g_o[0][i]
    unique_g_o[0][i] = str(tk_so_cl[1][tk_so_cl[0].index(str(unique_g_o[0][i]))])
    main_info[0][i] = unique_g_o[0][i]
    main_info[7][i] = str(unique_g_o[1][i])


# unique_g_o[0] = list(set(unique_g_o[0]))
# unique_g_o[1] = list(set((unique_g_o[1])))

# unique_g_o = pd.DataFrame(unique_g_o)

# unique_g_o = unique_g_o.transpose()
# unique_g_o.to_excel(unique_name, 
#                     index=False)

# print("""'unique' - файл с уникальными СО и товарами.
# Пожалуйста, выгрузите из системы SAP LRP актуальные ПЦ (без шаблона) и ЗЦ (шаблон для выгрузки: Maxim python).
# И назовите файлы 'пц' и 'зц' соответственно.""")

#заполняем основной массив дальше
print("...Импорт основной информации...")
print (str(time.time() - start))
main_info_1 = pd.read_excel(cluster_name,
                     skiprows=1,
                     usecols='A, F, G, J, K, P, R, T, U',
                     dtype=str)

main_info_1 = main_info_1.transpose()
main_info_1 = main_info_1.values.tolist()

for i in range(goods_number):
    main_info[6][i] = main_info_1[0][i]
    main_info[5][i] = int(main_info_1[1][i])
    main_info[4][i] = main_info_1[2][i]
    main_info[2][i] = main_info_1[3][i]
    main_info[3][i] = main_info_1[4][i]
    main_info[8][i] = main_info_1[5][i]
    main_info[74][i] = main_info_1[6][i]
    main_info_1[7][i] = str(main_info_1[7][i])
    main_info[17][i] = float(main_info_1[8][i])
    
    
### перенос в первый кластер для top-p1   
for i in range(goods_number):
    if (str(main_info[74][i]) == "TOP_P1" and
        str(main_info[6][i][main_info[6][i].find("_ГМ_"):]) != "_ГМ_1"):
        for j in range(len(tk_so_cl[0])):
            if str(main_info[6][i][:main_info[6][i].find("_ГМ_")]) + "_ГМ_1" == str(tk_so_cl[6][j]):
                main_info[6][i] = tk_so_cl[6][j]
                main_info[1][i] = tk_so_cl[3][j]
                


print("...Импорт конкурентов...")
print (str(time.time() - start))
main_info_2 = pd.read_excel(cluster_name,
                     skiprows=0,
                     usecols='AI:AWZ',
                     dtype=str)

main_info_2 = main_info_2.transpose()
main_info_2 = main_info_2.values.tolist()

main_info_2_1 = pd.read_excel(cluster_name,
                     skiprows=0,
                     nrows=1,
                     usecols='AI:AWZ',
                     dtype=str)

main_info_2_1 = main_info_2_1.columns.tolist()


#ИМПОРТ: пц
##################################################################

# print("Если пц, зц и бу выгружены, введите 1 для продолжения")
# one_to_continue = input()
# if (one_to_continue == 1):
#     pass

print("...Поиск пц...")
print (str(time.time() - start))
selling_prices = pd.read_excel(sp_name,
                     skiprows=0,
                     usecols='A, D, H, J, M, O, P, Q, T, U, V, BD',
                     dtype=str)

selling_prices = selling_prices.values.tolist()
selling_prices = sorted(selling_prices, key=itemgetter(0, 5), reverse=True)
selling_prices = pd.DataFrame(selling_prices)
selling_prices = selling_prices.transpose()
selling_prices = selling_prices.values.tolist()

for i in range(len(selling_prices[0])):
    if (math.isnan(float(selling_prices[4][i])) == False):
        selling_prices[4][i] = int(selling_prices[4][i])
    if ( (str(selling_prices[4][i]) == "nan") and (str(selling_prices[3][i]) == "nan") and
        (str(selling_prices[2][i]) == "nan") ):
        pass
    else:
        selling_prices[1][i] = -1


#################################################################################################################
start_from = 0
end_at = 0
selling_prices_0 = 0
selling_prices_00 = 0
sp_index = 0

for i in range(len(selling_prices[0])):
    selling_prices[0][i] = str(selling_prices[0][i])[6:]


for i in range(goods_number):
    try:
        start_from = selling_prices[0].index(str(main_info[7][i])[6:])
        end_at = start_from + selling_prices[0].count(str(main_info[7][i])[6:])
        try:
            sp_index = selling_prices[4].index(main_info[1][i], 
                                               start_from, 
                                               end_at)
            if len(str(selling_prices[8][sp_index])) > 3:
                main_info[10][i] = float(selling_prices[10][sp_index])
                main_info[12][i] = str(selling_prices[8][sp_index])
                main_info[65][i] = str(selling_prices[9][sp_index][:10])
                main_info[67][i] = str(selling_prices[6][sp_index])
                main_info[69][i] = float(selling_prices[7][sp_index])
                main_info[70][i] = str(selling_prices[11][sp_index])
            else:
                main_info[10][i] = float(selling_prices[7][sp_index])
                main_info[12][i] = str(selling_prices[8][sp_index])
                main_info[65][i] = str(selling_prices[9][sp_index][:10])
                main_info[67][i] = str(selling_prices[6][sp_index])
                main_info[69][i] = float(selling_prices[7][sp_index])
        except Exception:
            try:
                sp_index = selling_prices[3].index(main_info[6][i], 
                                                   start_from, 
                                                   end_at)
                if len(str(selling_prices[8][sp_index])) > 3:
                    main_info[10][i] = float(selling_prices[10][sp_index])
                    main_info[12][i] = str(selling_prices[8][sp_index])
                    main_info[65][i] = str(selling_prices[9][sp_index][:10])
                    main_info[67][i] = str(selling_prices[6][sp_index])
                    main_info[69][i] = float(selling_prices[7][sp_index])
                    main_info[70][i] = str(selling_prices[11][sp_index])
                else:
                    main_info[10][i] = float(selling_prices[7][sp_index])
                    main_info[12][i] = str(selling_prices[8][sp_index])
                    main_info[65][i] = str(selling_prices[9][sp_index][:10])
                    main_info[67][i] = str(selling_prices[6][sp_index])
                    main_info[69][i] = float(selling_prices[7][sp_index])
            except Exception:
                try:
                    sp_index = selling_prices[2].index(main_info[6][i][0:main_info[6][i].find("_ГМ_")+3],
                                                       start_from, 
                                                       end_at)
                    if len(str(selling_prices[8][sp_index])) > 3:
                        main_info[10][i] = float(selling_prices[10][sp_index])
                        main_info[12][i] = str(selling_prices[8][sp_index])
                        main_info[65][i] = str(selling_prices[9][sp_index][:10])
                        main_info[67][i] = str(selling_prices[6+1][sp_index])
                        main_info[69][i] = float(selling_prices[7][sp_index])
                        main_info[70][i] = str(selling_prices[11][sp_index])
                    else:
                        main_info[10][i] = float(selling_prices[7][sp_index])
                        main_info[12][i] = str(selling_prices[8][sp_index])
                        main_info[65][i] = str(selling_prices[9][sp_index][:10])
                        main_info[67][i] = str(selling_prices[6][sp_index])
                        main_info[69][i] = float(selling_prices[7][sp_index])
                except Exception:
                    try:
                        sp_index = selling_prices[1].index(main_info[0][i], 
                                                           start_from, 
                                                           end_at)
                        if len(str(selling_prices[8][sp_index])) > 3:
                            main_info[10][i] = float(selling_prices[10][sp_index])
                            main_info[12][i] = str(selling_prices[8][sp_index])
                            main_info[65][i] = str(selling_prices[9][sp_index][:10])
                            main_info[67][i] = str(selling_prices[6][sp_index])
                            main_info[69][i] = float(selling_prices[7][sp_index])
                            main_info[70][i] = str(selling_prices[11][sp_index])
                        else:
                            main_info[10][i] = float(selling_prices[7][sp_index])
                            main_info[12][i] = str(selling_prices[8][sp_index])
                            main_info[65][i] = str(selling_prices[9][sp_index][:10])
                            main_info[67][i] = str(selling_prices[6][sp_index])
                            main_info[69][i] = float(selling_prices[7][sp_index])
                    except Exception:
                        try:
                            sp_index = start_from
                            if len(str(selling_prices[8][sp_index])) > 3:
                                main_info[10][i] = float(selling_prices[10][sp_index])
                                main_info[12][i] = str(selling_prices[8][sp_index])
                                main_info[65][i] = str(selling_prices[9][sp_index][:10])
                                main_info[67][i] = str(selling_prices[6][sp_index])
                                main_info[69][i] = float(selling_prices[7][sp_index])
                                main_info[70][i] = str(selling_prices[11][sp_index])
                            else:
                                main_info[10][i] = float(selling_prices[7][sp_index])
                                main_info[12][i] = str(selling_prices[8][sp_index])
                                main_info[65][i] = str(selling_prices[9][sp_index][:10])
                                main_info[67][i] = str(selling_prices[6][sp_index])
                                main_info[69][i] = float(selling_prices[7][sp_index])
                        except Exception:
                            
                            selling_prices_0 = selling_prices_0 + 1
                            main_info[10][i] = -1
    except Exception:
        selling_prices_00 = selling_prices_00 + 1
        main_info[10][i] = -2


if (selling_prices_0 > 0):
    print("Ни по одной сцепке не найдено подходящих пц (товар есть в файле, -1): ", selling_prices_0)

if (selling_prices_00 > 0):
    print("Товар не найден в файле пц (-2): ", selling_prices_00)

#print("Кол-во пропусков в пц: ", selling_prices_0)




#################################################################################################################


#ИМПОРТ: зц
##################################################################
print("...Поиск зц...")
print (str(time.time() - start))



purchasing_prices = pd.read_excel(pp_name,
                     skiprows=0,
                     usecols='A, B, E, F, H, I, J, N, O, P, R, T, U, X',
                     dtype=str)

purchasing_prices = purchasing_prices.transpose()
purchasing_prices = purchasing_prices.values.tolist()
for i in range(len(purchasing_prices[1])):
    if (len(str(purchasing_prices[10][i])) > 3 ):
        pass
    else:
        purchasing_prices[10][i] = str("1")

purchasing_prices = pd.DataFrame(purchasing_prices)
purchasing_prices = purchasing_prices.transpose()
purchasing_prices = purchasing_prices.values.tolist()  
purchasing_prices = sorted(purchasing_prices, key=itemgetter(1, 12), reverse=True)
for i in range(len(purchasing_prices)):
    purchasing_prices[i].append(0)
purchasing_prices = pd.DataFrame(purchasing_prices)
purchasing_prices = purchasing_prices.transpose()
purchasing_prices = purchasing_prices.values.tolist()


for i in range(len(purchasing_prices[1])):
    if str(purchasing_prices[9][i]) == "Z2":
        if str(purchasing_prices[10][i]) == "RUB":
            purchasing_prices[14][i] = float(purchasing_prices[4][i])
        elif str(purchasing_prices[10][i]) == "EUR":
            purchasing_prices[14][i] = float(purchasing_prices[4][i]) * 81
        elif str(purchasing_prices[10][i]) == "CNY":
            purchasing_prices[14][i] = float(purchasing_prices[4][i]) * 11
        else:
            purchasing_prices[14][i] = float(purchasing_prices[4][i]) * 75
        
    elif (float(purchasing_prices[5][i]) == 0):
        if (str(purchasing_prices[9][i]) == "P5"):
            purchasing_prices[14][i] = float(purchasing_prices[4][i]) * (1 + 0.1)   
        else:
            purchasing_prices[14][i] = float(purchasing_prices[4][i]) * (1 + 0.2)
    else:
        if (str(purchasing_prices[9][i]) == "P5"):
            purchasing_prices[14][i] = float(purchasing_prices[5][i]) * (1 + 0.1)
        else:
            purchasing_prices[14][i] = float(purchasing_prices[5][i]) * (1 + 0.2)

for i in range(len(purchasing_prices[1+0])):
    if (purchasing_prices[14][i] == 0) or (purchasing_prices[14][i] == 1.1) or (purchasing_prices[14][i] == 1.2):
        purchasing_prices[1][i] = -1
        purchasing_prices[2][i] = -1
        purchasing_prices[4][i] = -1
        purchasing_prices[10][i] = -1
        purchasing_prices[7][i] = -1
        purchasing_prices[3][i] = -1
        purchasing_prices[5][i] = -1
        purchasing_prices[14][i] = -3
#############################################################################################################
start_from = 0
end_at = 0
purchasing_prices_0 = 0
purchasing_prices_00 = 0
pp_index = 0



for i in range(len(purchasing_prices[1])):
    purchasing_prices[1][i] = str(purchasing_prices[1][i])[6:]

for i in range(goods_number):
    try:
        start_from = purchasing_prices[1].index(str(main_info[7][i])[6:])
        end_at = start_from + purchasing_prices[1].count(str(main_info[7][i])[6:])
        try:
            pp_index = purchasing_prices[3].index(str(tk_so_cl[4][tk_so_cl[0].index(str(main_info[1][i]))]), 
                                                    start_from, 
                                                    end_at)
            main_info[9][i] = float(purchasing_prices[14][pp_index])
            main_info[11][i] = (main_info[10][i] - main_info[9][i]) / main_info[10][i]
            main_info[62][i] = str(purchasing_prices[0][pp_index])
        except Exception:
            try:
                pp_index = purchasing_prices[2].index(main_info[0][i], 
                                                        start_from, 
                                                        end_at)
                main_info[9][i] = float(purchasing_prices[14][pp_index])
                main_info[11][i] = (main_info[10][i] - main_info[9][i]) / main_info[10][i]
                main_info[62][i] = str(purchasing_prices[0][pp_index])
            except Exception:
                try:
                    pp_index = start_from
                    main_info[9][i] = float(purchasing_prices[14][pp_index])
                    main_info[11][i] = (main_info[10][i] - main_info[9][i]) / main_info[10][i]
                    main_info[62][i] = str(purchasing_prices[0][pp_index])
                except Exception:
                    purchasing_prices_0 = purchasing_prices_0 + 1
                    main_info[9][i] = -1
    except Exception:
        purchasing_prices_00 = purchasing_prices_00 + 1
        main_info[9][i] = -2


if (purchasing_prices_0 > 0):
    print("Ни по одной сцепке не найдено подходящих зц (товар есть в файле, -1): ", purchasing_prices_0)

if (purchasing_prices_00 > 0):
    print("Товар не найден в файле зц (-2): ", purchasing_prices_00)


#print("Кол-во пропусков в зц: ", purchasing_prices_0)




#ИМПОРТ: бу
##################################################################

print("...Поиск бу...")
print (str(time.time() - start))
try:
    bc = pd.read_excel(bc_name,
                         skiprows=0,
                         usecols='A, E, AD',
                         dtype=str)

    bc = bc.transpose()
    bc = bc.values.tolist()

    for i in range(goods_number):
        for j in range(len(bc[0])):
            if (str(main_info[7][i]) == str(bc[0][j]) and
                int(main_info[1][i]) == int(bc[1][j]) and
                (str(bc[2][j]).find("Каталог") != -1 or str(bc[2][j]).find("К-г") != -1)):
                    main_info[66][i] = 1
                    if len(str(main_info[67][i])) > 3:
                        main_info[66][i] = 2
                        if str(bc[2][j]).find("Каталог") != -1 or str(bc[2][j]).find("К-г") != -1:
                            main_info[12][i] = bc[2][j]  
                        continue
                    else:
                        main_info[66][i] = 1
                        if str(bc[2][j]).find("Каталог") != -1 or str(bc[2][j]).find("К-г") != -1:
                            main_info[12][i] = bc[2][j]
                        continue
except Exception:
    print("файл с бу отсутствует")
                

############################################################################################################

# Пилоты
##################################################################

top_1000_list = ['ТОП-1000', 'TOP_ONLINE', 'TOP_YARCHE', 'TOP_RAZ', 'TOP_MARIA', 'TOP_GULLIVER', 
                 'TOP_SMART', 'TOP_POKUPALKO', 'TOP_POBEDA', 'TOP1000_KB', 'TOP_ALCOHOL', 'TOP_7OFF']
ff_sc = ["P01", "P07", "P09", "P17", "P20", "P30", "P39", "P40", "P42", "P47", "P48", "P51"]
pilot_1_cities = ["Ярославль", "Новосибирск", "Нижний Новгород", "Санкт-Петербург", 
                  "Волгоград", "Волжский", "Новороссийск", "Краснодар"]
pilot_1_markets = ["МАГНИТ", "ПЯТЕРОЧКА", "ПОКУПАЛКО", "МАРИЯ-РА", "ЯРЧЕ"]
sc_pilot_exclude = ['P05', 'P06', 'P09', 'P12', 'P20', 'P21', 'P25', 'P36', 'P40', 'P48', 'P51']

for i in range(goods_number):
    
    if (main_info[7][i] == "580404052744" or
          main_info[7][i] == "580404226168"):
        main_info[73][i] = "расценка угля"
        
#     if (str(main_info[74][i]) in top_1000_list and
#         str(main_info[6][i][:main_info[6][i].find("_ГМ_")]) in pilot_1_cities and
#         (len(str(main_info[12][i])) <= 3 or main_info[70][i] == "Z022")):
#         if (str(main_info[4][i]) in sc_pilot_exclude or 
#             str(main_info[62][i]) == "104050" or 
#             str(main_info[7][i]) in sku_exclude[0]):
#             continue
#         else:
#             main_info[73][i] = "пилот 1"
        
    if (str(main_info[74][i]) == "TOP_DM" or str(main_info[74][i]) == "TOP-4500_BILLA"):
        if (str(main_info[4][i]) == "P41" or
            str(main_info[7][i])[:4] == "3803" or
            str(main_info[7][i])[:4] == "4401"):
            main_info[73][i] = "минимальная цена (рпц+апц)"
        else:
            main_info[73][i] = "медиана (рпц с рпц, апц с апц)"

        
        
        
        

############################################################################################################

# Расценка (для top-P1 и доп мониторингов)
##################################################################

for i in range(goods_number):
    if str(main_info[74][i]) == "TOP_DM" or str(main_info[74][i]) == "TOP-4500_BILLA":
        if str(main_info[4][i]) == "P36" or str(main_info[4][i]) == "P43":
            main_info[75][i] = "основные города"
        else:
            main_info[75][i] = "все СО"
    if str(main_info[74][i]) == "P1 NF_ON" or str(main_info[74][i]) == "P1 NF_OF":
        main_info[75][i] = "в 1 СО"
    if str(main_info[74][i]) == "NF_CL_ON" or str(main_info[74][i]) == "NF_CL_OF":
        main_info[75][i] = "единая цена в городе"
    if ((str(main_info[74][i]) == "TOP_P1" or str(main_info[74][i]) in top_1000_list) and
        str(main_info[6][i][:main_info[6][i].find("_ГМ_")]) != "Пермь"):
        main_info[75][i] = "единая цена в городе"

                        
    
############################################################################################################

#ИМПОРТ: цс
##################################################################
print("...Поиск цс...")
print (str(time.time() - start))

pf = pd.read_excel("(обязательно).xlsx",
                     sheet_name="ЦС",
                     skiprows=0,
                     usecols='A:C',
                     dtype=str)

pf = pf.transpose()
pf = pf.values.tolist()


for i in range(goods_number):
    for j in range(len(pf[0])):
        if str(main_info[7][i]) == str(pf[0][j]):
            main_info[71][i] = str(pf[1][j])
#             main_info[73][i] = "пилот ЦС"



############################################################################################################



#ниже проверяем, если цена совпадает с миниималкой, то пропускаем
########################################################
print("...Импорт для скрытого промо...")
print (str(time.time() - start))

hp_if = pd.read_excel("(обязательно).xlsx",
                     sheet_name="Скрытое промо",
                     skiprows=0,
                     usecols='A:B',
                     dtype=str)

hp_if = hp_if.transpose()
hp_if = hp_if.values.tolist()


hp_file_to_sap = [0] * 6
for i in range(6):
    hp_file_to_sap[i] = [0] * goods_number

main_info_3 = pd.read_excel(cluster_name,
                     skiprows=1,
                     usecols='R, S',
                     dtype=str)

main_info_3 = main_info_3.transpose()
main_info_3 = main_info_3.values.tolist()


hp_rival_to_code_sheet_name = str(week_number) + " " + "неделя" + " " + str(reaction_date[6:10])
hp_rival_to_code = pd.read_excel("Конкуренты_ТОП-1000_2023г.xlsx",
                     sheet_name=hp_rival_to_code_sheet_name,
                     skiprows=0,
                     usecols='G, I, J, L, P',
                     dtype=str)

hp_rival_to_code = hp_rival_to_code.transpose()
hp_rival_to_code = hp_rival_to_code.values.tolist()

print("...Расчет скрытого промо...")
print (str(time.time() - start))

for i in range(len(hp_rival_to_code[0])):
    try:
        hp_rival_to_code[2][i] = hp_rival_to_code[2][i].upper()
    except Exception:
        pass

for i in range(len(hp_rival_to_code[0])):
    if hp_rival_to_code[2][i] == "КРАСНОЕ И БЕЛОЕ":
        hp_rival_to_code[2][i] = "КРАСНОЕ"
    if hp_rival_to_code[2][i] == "КРАСНЫЙ ЯР":
        hp_rival_to_code[2][i] = "КРАСНЫЙ"
    if hp_rival_to_code[2][i] == "МАГНИТ КОСМЕТИК":
        hp_rival_to_code[2][i] = "МАГНИТ"
    if hp_rival_to_code[2][i] == "СПАР (SPAR)":
        hp_rival_to_code[2][i] = "СПАР"
    if hp_rival_to_code[2][i] == "ХЛЕБ СОЛЬ":
        hp_rival_to_code[2][i] = "ХЛЕБ"
        
        
        
    
    
#обычное заполнение цен конкурентов:


for i in range(goods_number):
    for j in range(0, len(main_info_2), 3):
        if (float(main_info_2[j][i+1]) > 0):
            
            if main_info[70][i] == "Z029" or main_info[70][i] == "Z039":
                continue
            if ((str(main_info[74][i]) == "TOP_DM" or
                 str(main_info[74][i]) == "TOP-4500_BILLA") and
                (str(main_info_2_1[j][0:main_info_2_1[j].find(" ")]) == "ЯНДЕКС" or
                 str(main_info_2_1[j][0:main_info_2_1[j].find(" ")]) == "WILDBERRIES")):
                continue
            if (len(str(main_info[12][i])) > 3 and
                main_info[73][i] == "медиана (рпц с рпц, апц с апц)"):
                continue
            main_info[19][i] += 1
            main_info[19+main_info[19][i]][i] = round(float(main_info_2[j][i+1]),2)
#             if (str(main_info[74][i]) == "TOP_P1"):
#                 continue
            main_info[19+20+main_info[19][i]][i] = str(main_info_2_1[j][0:main_info_2_1[j].find(" ")])



                        
############################################################################################################         
            
            
#добавляем к нашим апц апц конкурентов (заполнение цен)

for i in range(goods_number):
    if main_info[73][i] == "пилот 1":
        continue
    if (len(str(main_info[12][i])) <= 3 and
        main_info[73][i] != "минимальная цена (рпц+апц)" and
        str(main_info[2][i]) != "1"):          
        continue
        
    for j in range(1, len(main_info_2), 3):
        if (float(main_info_2[j][i+1]) > 0):
            if ((str(main_info[74][i]) == "TOP_DM" or
                 str(main_info[74][i]) == "TOP-4500_BILLA") and
                (str(main_info_2_1[j-1][0:main_info_2_1[j-1].find(" ")]) == "ЯНДЕКС" or
                 str(main_info_2_1[j-1][0:main_info_2_1[j-1].find(" ")]) == "WILDBERRIES")):
                continue
            main_info[19][i] = main_info[19][i] + 1
            if main_info[19][i] > 20:
                print("Превышение числа конкурентов! (заполнение цен), ", main_info[19][i])
                print(main_info[7][i])
                print(main_info[6][i])
            main_info[19+main_info[19][i]][i] = round(float(main_info_2[j][i+1]), 2)
#             if (str(main_info[74][i]) == "TOP_P1"):
#                 continue
            main_info[19+20+main_info[19][i]][i] = str(main_info_2_1[j-1][0:main_info_2_1[j-1].find(" ")])



    
    



#а здесь теперь все очищено от скрытого промо
#######################################################

#сначала убираем дубликаты:

for i in range (goods_number):
    for j in range(i+1, goods_number):
        if ((str(main_info[7][i])[0] != "(") and 
        (str(main_info[7][i]) == str(main_info[7][j])) and 
        (str(main_info[1][i]) == str(main_info[1][j]))):
            for l in range(main_info[19][j]):
                main_info[19][i] = main_info[19][i] + 1
                if main_info[19][i] > 20:
                    print("Превышение числа конкурентов! (проверка дубликатов), ", main_info[19][i])
                    print(main_info[7][i])
                    print(main_info[6][i])
                main_info[19+main_info[19][i]][i] = float(main_info[19+1+l][j])
#                 if (str(main_info[74][i]) == "TOP_P1"):
#                     continue
                main_info[19+20+main_info[19][i]][i] = str(main_info[19+20+1+l][j]) + "##" + str(main_info_3[0][j])
            main_info[7][j] = "(" + str(main_info[7][j]) + ")"
            main_info[1][j] = "(" + str(main_info[1][j]) + ")"


min_index = 0

for i in range(goods_number):
    if (main_info[19][i] > 0):
        find_min = [0] * main_info[19][i]
        for j in range(main_info[19][i]):
            find_min[j] = main_info[19+j+1][i]
        main_info[17][i] = min(find_min, key=lambda k: float(k))

only_one_hp = 0
iii = 0


hp_exclude_cities = ["Грозный", "Мурманск"]

for i in range(goods_number):
    only_one_hp = 0
    if (str(main_info[2][i]) == "1" or
        str(main_info[3][i]) == "+"):
        continue
    if str(main_info[6][i][:main_info[6][i].find("_ГМ_")]) in hp_exclude_cities:
        continue
    if main_info[73][i] == "пилот 1":
        continue
    if str(main_info[7][i]).find("(") != -1:
        continue
    if len(str(main_info[12][i])) > 3:
        continue
    for j in range(main_info[19][i]):
        if (float(main_info[19+1+j][i]) == float(main_info[17][i])) and (only_one_hp == 0):
            if (str(main_info[62][i]) == "101921"):
                if ((main_info[17][i] - main_info[9][i]) / main_info[17][i]) < -0.25:
                    only_one_hp = only_one_hp + 1
                    hp_file_to_sap[0][iii] = str(main_info_3[0][i])
                    hp_file_to_sap[1][iii] = str(main_info_3[1][i])
                    hp_file_to_sap[2][iii] = str(main_info[6][i])
                    hp_file_to_sap[3][iii] = str(main_info[19+1+j+20][i])
                    hp_file_to_sap[4][iii] = str(main_info[7][i])
                    hp_file_to_sap[5][iii] = round(main_info[17][i], 2)
                    iii = iii + 1
                    main_info[19+1+j][i] = "(" + str(main_info[19+1+j][i]) + ")"
                    main_info[19+1+j+20][i] = "(" + str(main_info[19+1+j+20][i]) + ")"
                else:
                    continue
            else:
                if str(main_info[4][i]) in hp_if[0]:
                    if (main_info[17][i] - main_info[9][i]) / main_info[17][i] < float(hp_if[1][hp_if[0].index(str(main_info[4][i]))]):
                        only_one_hp = only_one_hp + 1
                        hp_file_to_sap[0][iii] = str(main_info_3[0][i])
                        hp_file_to_sap[1][iii] = str(main_info_3[1][i])
                        hp_file_to_sap[2][iii] = str(main_info[6][i])
                        hp_file_to_sap[3][iii] = str(main_info[19+1+j+20][i])
                        hp_file_to_sap[4][iii] = str(main_info[7][i])
                        hp_file_to_sap[5][iii] = round(main_info[17][i], 2)
                        iii = iii + 1
                        main_info[19+1+j][i] = "(" + str(main_info[19+1+j][i]) + ")"
                        main_info[19+1+j+20][i] = "(" + str(main_info[19+1+j+20][i]) + ")"

                    else:
                        continue
                else:
                    if (main_info[17][i] - main_info[9][i]) / main_info[17][i] < float(0):
                        only_one_hp = only_one_hp + 1
                        hp_file_to_sap[0][iii] = str(main_info_3[0][i])
                        hp_file_to_sap[1][iii] = str(main_info_3[1][i])
                        hp_file_to_sap[2][iii] = str(main_info[6][i])
                        hp_file_to_sap[3][iii] = str(main_info[19+1+j+20][i])
                        hp_file_to_sap[4][iii] = str(main_info[7][i])
                        hp_file_to_sap[5][iii] = round(main_info[17][i], 2)
                        iii = iii + 1
                        main_info[19+1+j][i] = "(" + str(main_info[19+1+j][i]) + ")"
                        main_info[19+1+j+20][i] = "(" + str(main_info[19+1+j+20][i]) + ")"

                    else:
                        continue
        else:
            continue


##############################################################################################################
#доработка файла скрытого промо
#G - 0 - кластер (город)
#I - 1 - competitor
#J - 2 - конкурент
#L - 3 - тип мониторинга
#P - 4 - код конкурента

print("...Доработка файла скрытого промо...")
print (str(time.time() - start))

for i in range(len(hp_file_to_sap[0])):
    if (hp_file_to_sap[3][i] != 0) and (hp_file_to_sap[3][i].find("##") != -1):
        hp_file_to_sap[0][i] = hp_file_to_sap[3][i][hp_file_to_sap[3][i].find("##")+2:]
        hp_file_to_sap[3][i] = hp_file_to_sap[3][i][:hp_file_to_sap[3][i].find("##")]


for i in range(len(hp_file_to_sap[0])):
    if hp_file_to_sap[5][i] == 0:
        hp_file_to_sap[0][i] = ""
        hp_file_to_sap[1][i] = ""
        hp_file_to_sap[2][i] = ""
        hp_file_to_sap[3][i] = ""
        hp_file_to_sap[4][i] = ""
        hp_file_to_sap[5][i] = ""
    else:
        for j in range(len(hp_rival_to_code[0])):
            if (hp_rival_to_code[2][j] == hp_file_to_sap[3][i]) and (hp_rival_to_code[0][j] == hp_file_to_sap[2][i]):
                hp_file_to_sap[3][i] = "00000" + str(hp_rival_to_code[4][j])
                break

#После скрытого промо нужно вернуть конкурентов в основном массиве в нормальный вид

for i in range(goods_number):
#     if (str(main_info[74][i]) == "TOP_P1"):
#         continue
    if str(main_info[7][i]).find("(") == -1:
        for j in range(main_info[19][i]):
            if main_info[20+20+j][i].find("##") != -1:
                main_info[20+20+j][i] = main_info[20+20+j][i][:main_info[20+20+j][i].find("##")]


###############################################################################################################
#экспорт скрытого промо
print("...Экспорт скрытого промо...")
print (str(time.time() - start))

hp_file_to_sap = pd.DataFrame(hp_file_to_sap)
hp_file_to_sap = hp_file_to_sap.transpose()
hp_file_to_sap.to_excel(hp_name,
                    index=False,
                    header=["Тип мониторинга", "Дата мониторинга", "Кластер", "Код конкурента", "САП-код", "Цена"])
#пересчитываем кол-во конкурентов
rivals_number_re = 0

for i in range(goods_number):
    rivals_number_re = 0
    for j in range(main_info[19][i]):
        if (str(main_info[19+1+j][i]).find("(") != 0):
            rivals_number_re = rivals_number_re + 1
    main_info[19][i] = rivals_number_re
        

#################################################################
#заполняем мин рег цену конкурентов
print("...Расчет цен...")
print (str(time.time() - start))


min_index = 0


for i in range(goods_number):
    if (main_info[19][i] > 0):
        find_min = []
        for j in range(20):
            if (str(main_info[19+1+j][i]).find("(") != 0):
                if (main_info[19+1+j][i] > 0):
                    find_min.append(main_info[19+j+1][i])
        main_info[17][i] = min(find_min, key=lambda k: float(k))
        for l in range(20):
            if (str(main_info[19+1+l][i]).find("(") != 0):
                if (main_info[19+1+l][i] == main_info[17][i]):
                    main_info[18][i] = main_info[19+1+l+20][i]
                    break
        main_info[76][i] = statistics.mean(find_min)
        main_info[60][i] = statistics.median(find_min)
        find_min.sort()
        if (main_info[19][i] >= 2):
            main_info[61][i] = find_min[1]
        else:
            main_info[61][i] = 0
    else:
        main_info[17][i] = 0

        
        
# Пилоты 2
##################################################################     
        
# for i in range(goods_number):
#     if ((str(main_info[18][i]) not in pilot_1_markets) and main_info[73][i] == "пилот 1"):
#         main_info[73][i] = 0
        
for i in range(goods_number):
    if ((str(main_info[18][i]) == "КРАСНОЕ") or
        (str(main_info[18][i]) == "СЕМИШАГОФФ") or
        (str(main_info[18][i]) == "СМАРТ") or
        (str(main_info[18][i]) == "ЧИЖИК")):
        main_info[73][i] = "Дискаунтеры"
    

################################################################## 
#ЦО

#правило 1 (Основное ЦО)


def price_calc(price, comment):
    main_info[13][i] = price
    main_info[14][i] = (main_info[13][i] - main_info[9][i]) / main_info[13][i]
    main_info[15][i] = (main_info[13][i] - main_info[10][i]) / main_info[10][i]
    if main_info[77][i] == 0:
        main_info[77][i] = comment + str(round(main_info[13][i],2))
    else:
        main_info[77][i] = str(main_info[77][i]) + comment + str(round(main_info[13][i],2))

for i in range(goods_number):
    if (main_info[19][i] == 0):
        if (str(main_info[2][i]) == "1" and
            main_info[11][i] < 0.1 and
            len(str(main_info[12][i])) <= 3):
            price_calc((main_info[9][i] * 1.1), "повышение до м1=10% ")
        else:
            price_calc((main_info[10][i]), "тек цена ")
    else:
        if (str(main_info[6][i][:main_info[6][i].find("_ГМ_")]) == "Грозный "):
            price_calc((main_info[17][i] - 1), "-1 руб ")
                
        elif (main_info[73][i] == "пилот 1"):
            if ((str(main_info[4][i]) == "P06" or str(main_info[4][i]) == "P25")):
                price_calc((main_info[17][i] * 0.85), "-15% ")
            else:
                price_calc((main_info[17][i] * 0.90), "-10% ")                           
        elif (main_info[70][i] == "Z029"):
            price_calc((main_info[17][i] * 1.05), "+5% ")
        elif (main_info[71][i] != 0 and 
              str(main_info[2][i]) != "1" and
              main_info[19][i] > 1):
            if len(str(main_info[12][i])) <= 3:
                price_calc(((main_info[17][i] + main_info[61][i]) / 2), "ср между 1 и 2 к ")
            else:
                if (main_info[76][i] - main_info[17][i]) / main_info[17][i] > 0.05:
                    price_calc((main_info[17][i] * 1.05), "+5% ср цена намного выше мин ")
                else:
                    price_calc((main_info[76][i]), "ср цена ")      
        else:
            if (str(main_info[2][i]) == "1"  and main_info[19][i] > 1 and 
                (main_info[4][i] == "P09" or main_info[4][i] == "P51") and
                ((main_info[61][i] - main_info[17][i]) / main_info[17][i] > 0.3)):
                price_calc((main_info[61][i] - 0.01), "-1 коп под 2 к ")
            else:
                price_calc((main_info[17][i] - 0.01), "-1 коп ")
                       
                            
#правило 2 (Ограничение по максимальному отклонению вверх)

for i in range(goods_number):
    if (main_info[70][i] == "Z029" or main_info[70][i] == "Z039"):
        continue
    elif (main_info[73][i] == "пилот 1" and main_info[15][i] > 0.05):
        price_calc((main_info[10][i] * 1.05), " -> огр по откл 5% ")                   
    elif str(main_info[2][i]) == "1":
        if main_info[19][i] == 1:
            if (str(main_info[4][i]) == "P06" and main_info[15][i] > 0.2):
                price_calc((main_info[10][i] * 1.2), " -> огр по откл 20% ")
            elif main_info[15][i] > 0.1:
                price_calc((main_info[10][i] * 1.1), " -> огр по откл 10% ")
    elif ((main_info[19][i] == 1) and main_info[15][i] > 0.1):
        price_calc((main_info[10][i] * 1.1), " -> огр по откл 10% ")
    elif (main_info[15][i] > 0.2):
        price_calc((main_info[10][i] * 1.2), " -> огр по откл 20% ")
          



#правило 3 (Ограничение по максимальной м1 с детализацией (Подкатегория))

for i in range(goods_number):
    if main_info[15][i] > 0:
        if main_info[19][i] == 1:
            if str(main_info[3][i]) == "+":
                for j in range(len(avg_m1_pl[0])):
                    if ((str(main_info[5][i]) == str(avg_m1_pl[0][j])) and
                        (str(main_info[0][i]) == str(avg_m1_pl[2][j])) and
                        (float(main_info[14][i]) > float(avg_m1_pl[1][j]))):
                        if ((math.trunc(main_info[9][i] / (1 - float(avg_m1_pl[1][j])) * 10) /
                             10 - 0.01) <= main_info[10][i]):
                            price_calc(main_info[10][i], " -> тек цена из-за снижения по ср м1 (" + 
                                       str(round(float(avg_m1_pl[1][j]) * 100, 1)) + "%) ")
                        else:
                            price_calc((main_info[9][i] / (1 - float(avg_m1_pl[1][j]))), 
                                       " -> огр по ср м1 (" + str(round(float(avg_m1_pl[1][j]) * 100, 1)) + "%) ")                            
            else:
                for j in range(len(avg_m1_br[0])):
                    if ((str(main_info[5][i]) == str(avg_m1_br[0][j])) and
                        (str(main_info[0][i]) == str(avg_m1_br[2][j])) and
                        (float(main_info[14][i]) > float(avg_m1_br[1][j]))):
                        if ((math.trunc(main_info[9][i] / (1 - float(avg_m1_br[1][j])) * 10) /
                             10 - 0.01) <= main_info[10][i]): 
                            price_calc(main_info[10][i], " -> тек цена из-за снижения по ср м1 (" + 
                                       str(round(float(avg_m1_br[1][j]) * 100, 1)) + "%) ")
                        else:
                            price_calc((main_info[9][i] / (1 - float(avg_m1_br[1][j]))), 
                                       " -> огр по ср м1 (" + str(round(float(avg_m1_br[1][j]) * 100, 1)) + "%) ") 
        else:
            if str(main_info[3][i]) == "+":
                for j in range(len(max_m1_pl[0])):
                    if ((str(main_info[5][i]) == str(max_m1_pl[0][j])) and
                        (float(main_info[14][i]) > float(max_m1_pl[1][j]))):
                        if ((math.trunc(main_info[9][i] / (1 - float(max_m1_pl[1][j])) * 10) /
                             10 - 0.01) <= main_info[10][i]):
                            price_calc(main_info[10][i], " -> тек цена из-за снижения по макс м1 (" + 
                                       str(round(float(max_m1_pl[1][j]) * 100, 1)) + "%) ")
                        else:
                            price_calc((main_info[9][i] / (1 - float(max_m1_pl[1][j]))), 
                                       " -> огр по макс м1 (" + str(round(float(max_m1_pl[1][j]) * 100, 1)) + "%) ")
            else:
                for j in range(len(max_m1_br[0])):
                    if ((str(main_info[5][i]) == str(max_m1_br[0][j])) and
                        (float(main_info[14][i]) > float(max_m1_br[1][j]))):
                        
                        if ((math.trunc(main_info[9][i] / (1 - float(max_m1_br[1][j])) * 10) /
                             10 - 0.01) <= main_info[10][i]):
                            price_calc(main_info[10][i], " -> тек цена из-за снижения по макс м1 (" + 
                                       str(round(float(max_m1_br[1][j]) * 100, 1)) + "%) ")
                        else:
                            price_calc((main_info[9][i] / (1 - float(max_m1_br[1][j]))), 
                                       " -> огр по макс м1 (" + str(round(float(max_m1_br[1][j]) * 100, 1)) + "%) ")
                            
# правило 4 (макс цена):
    
for i in range(goods_number):                                 
    if (str(main_info[7][i]) == str("350901587447") and
        main_info[13][i] > 109.99):
        price_calc(109.99, " -> огр по макс цене (109.99) ")

        


#правило 5 (округление)


for i in range(goods_number):
    if main_info[13][i] == main_info[10][i]:
        continue
    if str(main_info[2][i]) == "1":
        if main_info[13][i] < 50:
            main_info[13][i] = main_info[13][i] // 1  + 0.99
            main_info[77][i] = str(main_info[77][i]) + " -> округление вверх ХХ.99 "  + str(round(main_info[13][i],2))
        else:
            if (main_info[17][i] != 0 and
                ((main_info[13][i] // 5 * 5 + 4.99) - main_info[17][i]) / main_info[17][i] > 0.05):
                main_info[13][i] = main_info[13][i] // 5 * 5 - 0.01
                main_info[77][i] = str(main_info[77][i]) + " -> округление вниз Х4.99 "  + str(round(main_info[13][i],2))
            else:
                main_info[13][i] = main_info[13][i] // 5 * 5 + 4.99
                main_info[77][i] = str(main_info[77][i]) + " -> округление вверх Х4.99 "  + str(round(main_info[13][i],2))
    elif main_info[73][i] == "Дискаунтеры":
        if main_info[13][i] < 100:
            main_info[13][i] = round(main_info[13][i] // 0.1 / 10 + 0.09, 2)
            main_info[77][i] = str(main_info[77][i]) + " -> округление вниз ХХ.Х9 "  + str(round(main_info[13][i],2))
        else:
            main_info[13][i] = main_info[13][i] // 5 * 5 + 4.99
            main_info[77][i] = str(main_info[77][i]) + " -> округление вверх Х4.99 "  + str(round(main_info[13][i],2))
    else:
        main_info[13][i] = round(main_info[13][i] // 0.1 / 10 - 0.01, 2)
        main_info[77][i] = str(main_info[77][i]) + " -> округление вниз ХХ.Х9 "  + str(round(main_info[13][i],2))

        
# правило 6 (МРЦ)

mrc= pd.read_excel("(обязательно).xlsx",
                     sheet_name="МРЦ",
                     skiprows=0,
                     usecols='A:B')
mrc = mrc.transpose()
mrc = mrc.values.tolist()

mrc_add= pd.read_excel("(обязательно).xlsx",
                     sheet_name="МРЦ доп",
                     skiprows=0,
                     usecols='A:D')
mrc_add = mrc_add.transpose()
mrc_add = mrc_add.values.tolist()

mrcpivo= pd.read_excel("(обязательно).xlsx",
                     sheet_name="МРЦ пиво",
                     skiprows=0,
                     usecols='A:B')
mrcpivo = mrcpivo.transpose()
mrcpivo = mrcpivo.values.tolist()

p12_cities = ["Альметьевск", "Казань", "Н.Челны", "Нижнекамск"]
p0433_bash = ["Уфа", "Стерлитамак"]
p0433_belg = ["Белгород"]

for i in range(goods_number):
    if (str(main_info[6][i][:main_info[6][i].find("_ГМ_")]) in p12_cities and str(main_info[4][i]) == "P12"):
        main_info[72][i] = "проверить МРЦ P12"
    if str(main_info[4][i]) == "P04" or str(main_info[4][i]) == "P33":
        main_info[72][i] = "МРЦ P04, P33"    

for i in range(goods_number):
     for j in range(len(mrc[0])):    
        if str(main_info[7][i]) == str(mrc[0][j]) and float(main_info[13][i]) < float(mrc[1][j]):
            price_calc(float(mrc[1][j]), str(main_info[77][i]) + "-> МРЦ (" + str(mrc[1][j]) + ") ")
            
for i in range(goods_number):       
    for j in range(len(mrc_add[0])):        
        if (str(main_info[7][i]) == str(mrc_add[0][j]) and float(main_info[13][i]) < float(mrc_add[1][j]) 
            and str(main_info[6][i][:main_info[6][i].find("_ГМ_")]) in p12_cities):
            price_calc(float(mrc_add[1][j]), str(main_info[77][i]) + "-> МРЦ Татарстан (" + float(mrc_add[1][j]) + ") ")
        if (str(main_info[7][i]) == str(mrc_add[0][j]) and float(main_info[13][i]) < float(mrc_add[2][j]) 
            and str(main_info[6][i][:main_info[6][i].find("_ГМ_")]) in p0433_bash):
            price_calc(float(mrc_add[2][j]), str(main_info[77][i]) + "-> МРЦ Башкортостан (" + float(mrc_add[2][j]) + ") ")
        if (str(main_info[7][i]) == str(mrc_add[0][j]) and float(main_info[13][i]) < float(mrc_add[3][j]) 
            and str(main_info[6][i][:main_info[6][i].find("_ГМ_")]) in p0433_belg):
            price_calc(float(mrc_add[3][j]), str(main_info[77][i]) + "-> МРЦ Белгород (" + float(mrc_add[3][j]) + ") ")
            
for i in range(goods_number):
     for j in range(len(mrcpivo[0])):        
        if (str(main_info[72][i]) == str("проверить МРЦ P12") and
            str(main_info[7][i]) == str(mrcpivo[0][j]) and
            float(main_info[13][i]) < float(mrcpivo[1][j])):
            price_calc(float(mrcpivo[1][j]), str(main_info[77][i]) + "-> МРЦ пиво (" + float(mrcpivo[1][j]) + ") ")
    
#правило 7 (комментарии)


# дубликаты
for i in range(goods_number):
    if (str(main_info[7][i]).find("(") == 0):
        main_info[16][i] = "Не меняем цену: дубликат"
        
# ограничение по минимальному проценту изменения
for i in range(goods_number):
    if main_info[16][i] != 0:
        continue
    elif ((main_info[70][i] == "Z029" or main_info[70][i] == "Z039") and
        (main_info[19][i] <= 1 or
         (main_info[61][i] - main_info[10][i]) / main_info[10][i] > 0.1 or
         (main_info[17][i] - main_info[10][i]) / main_info[10][i] < -0.1)):
        main_info[16][i] = "Не меняем цену: EDPP откл < 10%"
    elif str(main_info[2][i]) == "1":
        if (abs(main_info[15][i]) < 0.1 and
            str(main_info[4][i]) == "P06"):
            main_info[16][i] = "Не меняем цену: откл < 10%"
        elif abs(main_info[15][i]) < 0.05:
            main_info[16][i] = "Не меняем цену: откл < 5%"
    elif main_info[73][i] == "Дискаунтеры":
        if abs(main_info[15][i]) < 0.05:
            main_info[16][i] = "Не меняем цену: откл < 5%"
    elif str(main_info[74][i]) == "TOP_DM" or str(main_info[74][i]) == "TOP-4500_BILLA":
        if (abs(main_info[15][i]) < 0.03):
            main_info[16][i] = "Не меняем цену:  изменение меньше 3%"
    elif str(main_info[4][i]) in reaction_step[1]  and abs(main_info[15][i]) < 0.05:
        main_info[16][i] = "Не меняем цену: откл < 5%"
    elif (abs(main_info[15][i]) < 0.05 and str(main_info[4][i]) == "P06"):
        main_info[16][i] = "Не меняем цену: откл < 5%"
    elif (abs(main_info[15][i]) < 0.01) or (abs(main_info[13][i] - main_info[10][i]) < 1):
        main_info[16][i] = "Не меняем цену: откл < 1% или 1 рубля"
        
# нет реакции
for i in range(goods_number): 
    if main_info[16][i] != 0:
        continue
    elif (main_info[66][i] == 1):
        main_info[16][i] = "Не меняем цену: БУ из к-га"
    elif (main_info[70][i] == "Z033" or main_info[70][i] == "Z012" or
          main_info[70][i] == "Z009"):
        main_info[16][i] = "Не меняем апц: Z009, Z012, Z033"
    elif ((main_info[70][i] == "Z029" or main_info[70][i] == "Z039") and
          main_info[13][i] / main_info[69][i] > 0.99):
        main_info[16][i] = "Не меняем апц: сильное повышение EDPP"
            
    elif (len(str(main_info[12][i])) > 3 and
          main_info[15][i] > 0 and
          main_info[70][i] != "Z029" and main_info[70][i] != "Z039"):
        main_info[16][i] = "Не меняем апц: не повышаем апц"
    elif str(main_info[74][i]) == "TOP_DM" or str(main_info[74][i]) == "TOP-4500_BILLA":
        if (main_info[15][i] > 0):
            main_info[16][i] = "Не меняем цену: не повышаем цену"
        elif (main_info[60][i] - main_info[17][i]) / main_info[60][i] > 0.15:
            main_info[16][i] = "Не меняем цену: отклонение от медины более 15%"


# изменение цены
for i in range(goods_number): 
    if main_info[16][i] != 0:
        continue
    elif (main_info[2][i] == "1"):
        if len(str(main_info[12][i])) > 3:
            main_info[16][i] = "Меняем апц"
        else:
            if (main_info[14][i] < 0 and main_info[15][i] < 0):
                main_info[16][i] = "Заводим бу"
            else:
                main_info[16][i] = "Меняем рпц"    
    elif main_info[73][i] == "пилот 1":
        if main_info[70][i] == "Z022":
            if main_info[13][i] >= main_info[69][i]:
                main_info[16][i] = "Отменяем  апц"
            else:
                main_info[16][i] = "Меняем апц"
        else:
            if (float(main_info[15][i]) < - 0.05 and str(main_info[18][i]) in pilot_1_markets):
                main_info[16][i] = "Заводим апц до 13.07"
            else:
                if (main_info[14][i] < 0 and main_info[15][i] < 0 and str(main_info[4][i]) in ff_sc):
                    main_info[16][i] = "Заводим бу"
                else:
                    main_info[16][i] = "Меняем рпц"       
    elif len(str(main_info[12][i])) > 3:
        if ((str(main_info[4][i]) == "P12" or str(main_info[4][i]) == "P38") and main_info[15][i] < 0):
            main_info[16][i] = "Заводим бу"
        else: 
            main_info[16][i] = "Меняем апц"
    else:
        if (main_info[14][i] < 0 and main_info[15][i] < 0 and str(main_info[4][i]) in ff_sc):
            main_info[16][i] = "Заводим бу"
        else:
            main_info[16][i] = "Меняем рпц"
        
          
            
# for i in range(goods_number):
    
#     if (str(main_info[7][i]).find("(") == 0):
#         main_info[16][i] = "Не меняем цену: дубликат"
#     elif (len(str(main_info[12][i])) <= 3 and
#           main_info[66][i] == 1):
#         main_info[16][i] = "Не меняем рпц: БУ из к-га"
#     elif (len(str(main_info[12][i])) > 3 and
#           main_info[66][i] == 1):
#         main_info[16][i] = "Не меняем апц: БУ из к-га"
#     elif (main_info[70][i] == "Z033" or main_info[70][i] == "Z012" or
#           main_info[70][i] == "Z009"):
#         main_info[16][i] = "Не меняем апц: Z009, Z012, Z033"
#     elif ((main_info[70][i] == "Z029" or main_info[70][i] == "Z039") and
#           (main_info[19][i] == 1 or
#            abs(main_info[15][i]) < 0.1 or
#            abs((main_info[61][i] - main_info[10][i]) / main_info[10][i]) < 0.1)):
#         main_info[16][i] = "Не меняем апц: EDPP < 10%"
#     elif (len(str(main_info[12][i])) <= 3 and
#           main_info[19][i] == 0 and
#           str(main_info[74][i]) != "TOP_P1"):
#         main_info[16][i] = "Не меняем рпц: 0 кокурентов"
#     elif (len(str(main_info[12][i])) > 3 and
#           main_info[19][i] == 0 and
#           str(main_info[74][i]) != "TOP_P1"):
#         main_info[16][i] = "Не меняем апц: 0 кокурентов"
#     elif (len(str(main_info[12][i])) <= 3 and
#           main_info[75][i] == 0 and
#           str(main_info[74][i]) != "TOP_P1" and
#           ((abs(main_info[15][i]) < 0.01) or (abs(main_info[13][i] - main_info[10][i]) < 1) or
#            (abs(main_info[15][i]) < 0.05 and str(main_info[4][i]) == "P06"))):
#         main_info[16][i] = "Не меняем рпц: 1% или 1 рубль"
#     elif (len(str(main_info[12][i])) > 3 and
#           main_info[75][i] == 0 and
#           str(main_info[74][i]) != "TOP_P1" and
#           main_info[73][i] != "пилот 1" and
#           ((abs(main_info[15][i]) < 0.01) or (abs(main_info[13][i] - main_info[10][i]) < 1) or
#            (abs(main_info[15][i]) < 0.05 and str(main_info[4][i]) == "P06"))):
#         main_info[16][i] = "Не меняем апц: 1% или 1 рубль"
           
#     elif main_info[75][i] != 0:
        
#         if str(main_info[74][i]) == "TOP_DM" or str(main_info[74][i]) == "TOP-4500_BILLA":
#             if len(str(main_info[12][i])) > 3:
#                 if (main_info[15][i] > 0):
#                     main_info[16][i] = "Не меняем апц: отклонение больше 0"
#                 elif (abs(main_info[15][i]) < 0.03):
#                     main_info[16][i] = "Не меняем апц:  изменение меньше 3%"
#                 elif (main_info[60][i] - main_info[17][i]) / main_info[60][i] > 0.15:
#                     main_info[16][i] = "Не меняем апц: отклонение от медины более 15%"
#                 else:
#                     main_info[16][i] = "Меняем апц"
#             else:
#                 if main_info[15][i] > 0:
#                     main_info[16][i] = "Не меняем рпц: отклонение больше 0"
#                 elif (abs(main_info[15][i]) < 0.03):
#                     main_info[16][i] = "Не меняем рпц: изменение меньше 3%"
#                 elif (main_info[60][i] - main_info[17][i]) / main_info[60][i] > 0.15:
#                     main_info[16][i] = "Не меняем апц: отклонение от медины более 15%"
#                 else:
#                     main_info[16][i] = "Меняем рпц"
#         elif (str(main_info[74][i]) == "P1 NF_ON" or str(main_info[74][i]) == "P1 NF_OF"):
#             if len(str(main_info[12][i])) > 3:
#                 if (main_info[15][i] > 0 and
#                     main_info[70][i] != "Z029" and
#                     main_info[70][i] != "Z039"):
#                     main_info[16][i] = "Не меняем апц: не повышаем апц" 
#                 else:
#                     if ((main_info[70][i] == "Z029" or
#                         main_info[70][i] == "Z039") and
#                         main_info[13][i] / main_info[69][i] > 0.99):
#                         main_info[16][i] = "Не меняем апц: сильное повышение EDPP"
#                     else:
#                         main_info[16][i] = "Меняем апц"
#             else:
#                 main_info[16][i] = "Меняем рпц"
            
#         elif str(main_info[74][i]) == "TOP_P1": 
#             if (abs(main_info[15][i]) < 0.1 and
#                 str(main_info[4][i]) == "P06"):
#                 if len(str(main_info[12][i])) > 3:
#                     main_info[16][i] = "Не меняем апц: изменение меньше 10%"
#                 else:
#                     main_info[16][i] = "Не меняем рпц: изменение меньше 10%"   
            
#             elif (abs(main_info[15][i]) < 0.05):
#                 if len(str(main_info[12][i])) > 3:
#                     main_info[16][i] = "Не меняем апц: изменение меньше 5%"
#                 else:
#                     main_info[16][i] = "Не меняем рпц: изменение меньше 5%"
                    
                    
#             elif len(str(main_info[12][i])) > 3:
#                 if (main_info[15][i] > 0 and
#                     main_info[70][i] != "Z029" and
#                     main_info[70][i] != "Z039"):
#                     main_info[16][i] = "Не меняем апц: не повышаем апц" 
#                 else:
#                     if ((main_info[70][i] == "Z029" or
#                         main_info[70][i] == "Z039") and
#                         main_info[13][i] / main_info[69][i] > 0.99):
#                         main_info[16][i] = "Не меняем апц: сильное повышение EDPP"
#                     else:
#                         main_info[16][i] = "Меняем апц"
#             else:
#                 if (main_info[14][i] < 0 and
#                     main_info[15][i] < 0):
#                     main_info[16][i] = "Заводим бу"
#                 else:
#                     main_info[16][i] = "Меняем рпц"
            
#         elif str(main_info[74][i]) == "NF_CL_ON" or str(main_info[74][i]) == "NF_CL_OF":
#             if len(str(main_info[12][i])) > 3:
#                 if (main_info[15][i] > 0 and
#                     main_info[70][i] != "Z029" and
#                     main_info[70][i] != "Z039"):
#                     main_info[16][i] = "Не меняем апц: не повышаем апц" 
#                 else:                   
#                     if ((main_info[70][i] == "Z029" or
#                         main_info[70][i] == "Z039") and
#                         main_info[13][i] / main_info[69][i] > 0.99):
#                         main_info[16][i] = "Не меняем апц: сильное повышение EDPP"
#                     else:
#                         main_info[16][i] = "Меняем апц"
#             else:
#                 main_info[16][i] = "Меняем рпц"
                
#     elif main_info[73][i] == "пилот 1":
#         if ( len(str(main_info[12][i])) > 3 and
#               (abs(main_info[13][i] - main_info[10][i]) < 0.01)):
#             main_info[16][i] = "Не меняем апц: 1% или 1 рубль"
#         elif main_info[70][i] == "Z022":
#             if main_info[15][i] > 0:
#                 if main_info[13][i] >= main_info[69][i]:
#                     main_info[16][i] = "Отменяем  апц"
#                 else:
#                     main_info[16][i] = "Меняем апц"
#             else:
#                 main_info[16][i] = "Меняем апц"
#         elif len(str(main_info[12][i])) <= 3:
#             if (float(main_info[15][i]) < - 0.05 and
#                 str(main_info[18][i]) in pilot_1_markets):
#                 main_info[16][i] = "Заводим апц до 29.06"
#             else:
#                 if (main_info[14][i] < 0 and
#                     main_info[15][i] < 0 and
#                     str(main_info[4][i]) in ff_sc):
#                     main_info[16][i] = "Заводим бу"
#                 else:
#                     main_info[16][i] = "Меняем рпц"

#         else:
#             if (main_info[15][i] > 0 and
#                 main_info[70][i] != "Z029" and
#                 main_info[70][i] != "Z039"):
#                 main_info[16][i] = "Не меняем апц: не повышаем апц"
#             else:
#                 if ((main_info[70][i] == "Z029" or
#                     main_info[70][i] == "Z039") and
#                     main_info[13][i] / main_info[69][i] > 0.99):
#                     main_info[16][i] = "Не меняем апц: сильное повышение EDPP"
#                 else:
#                     if ((str(main_info[4][i]) == "P12" or str(main_info[4][i]) == "P38") and 
#                         main_info[15][i] < 0):
#                         main_info[16][i] = "Заводим бу"
#                     else: 
#                         main_info[16][i] = "Меняем апц"
    
#     elif len(str(main_info[12][i])) > 3:
#         if (main_info[15][i] > 0 and
#             main_info[70][i] != "Z029" and
#             main_info[70][i] != "Z039"):
#             main_info[16][i] = "Не меняем апц: не повышаем апц"
#         else:
#             if ((main_info[70][i] == "Z029" or
#                 main_info[70][i] == "Z039") and
#                 main_info[13][i] / main_info[69][i] > 0.99):
#                 main_info[16][i] = "Не меняем апц: сильное повышение EDPP"
#             else:
#                 if ((str(main_info[4][i]) == "P12" or str(main_info[4][i]) == "P38") and 
#                     main_info[15][i] < 0):
#                     main_info[16][i] = "Заводим бу"
#                 else: 
#                     main_info[16][i] = "Меняем апц"
   
#     else:
#         if (main_info[14][i] < 0 and
#             main_info[15][i] < 0 and
#             str(main_info[4][i]) in ff_sc):
#             main_info[16][i] = "Заводим бу"
#         else:
#             main_info[16][i] = "Меняем рпц"

###############################################################################



#кря-кря-кря


print("...Экспорт в эксель...")
print (str(time.time() - start))


main_info = pd.DataFrame(main_info)
main_info = main_info.transpose()
main_info.to_excel(final_name,
                    header=["СО", "ТК", "EPP", "ЧТМ", "ГЗ", "Подкатегория", "Город (кластер)", "Товар", 
                            "Наименование", "ЗЦ с НДС", "ПЦ текущая", "М1 текущая", "РА Ленты", "Цена итог", 
                            "М1 итог", "Отклонение итог", "Комментарий", "Мин цена конкурентов", 
                            "Сеть наименьшего конкурента", "Кол-во конкурентов без СкрП","1 конкурент", 
                            "2 конкурент", "3 конкурент", "4 конкурент", "5 конкурент", "6 конкурент", "7 конкурент",
                            "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20",
                            "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14",
                            "15", "16", "17", "18", "19", "20", "Медиана", "Второй минимальный", "Номер поставщика",
                            "Доля промо", "Рейтинг товара", "Дата начала цены (акция + 15 дней)",
                            "Прогрессивная акция","Код РА", "КЦ", "рпц", "тип ра", "цс", "МРЦ","Пилот",
                            "Монитоиринг", "Расценка", "Средняя цена", "Алгоритм"], 
                    index=False)


#zsd_prices


p36_p43_cities = ["Санкт-Петербург", "Москва", "Новосибирск", "Краснодар", "Екатеринбург"]


zsd_prices = []
for i in range(9):
    zsd_prices.append([])


for i in range(goods_number):
    if (main_info[16][i] == "Меняем рпц"):
        cluster_list = []
        sku_list = []
        price_list = []

        if (main_info[75][i] == "все СО"):
            cluster_list = list(set(tk_so_cl[2]))

        elif main_info[75][i] == "в 1 СО":
            for j in range(len(tk_so_cl[0])):
                if str(main_info[0][i]) == str(tk_so_cl[1][j]):
                    cluster_list.append(str(tk_so_cl[2][j]))
                cluster_list = list(set(cluster_list))
        elif main_info[75][i] == "основные города":
            for j in range(len(tk_so_cl[0])):
                if str(tk_so_cl[5][j]) in p36_p43_cities:
                    cluster_list.append(str(tk_so_cl[2][j]))
                cluster_list = list(set(cluster_list)) 
                
        elif main_info[75][i] == "единая цена в городе":
            for j in range(len(tk_so_cl[0])):
                if str(tk_so_cl[5][j]) == str(main_info[6][i][:main_info[6][i].find("_ГМ_")]):
                    cluster_list.append(str(tk_so_cl[2][j]))
                cluster_list = list(set(cluster_list))     
        else:
            cluster_list.append(str(tk_so_cl[2][tk_so_cl[0].index(str(main_info[1][i]))]))
            
        if main_info[71][i] != 0:
            
            for j in range(len(pf[0])):
                if str(main_info[71][i]) == pf[1][j]:
                    sku_list.append(str(pf[0][j]))
                sku_list = list(set(sku_list))
            for j in range(len(sku_list)):
                price_list.append(main_info[13][i])
        elif main_info[73][i] == "расценка угля":
            for j in range(len(coal_price[0])):
                if main_info[7][i] == coal_price[0][j]:
                    base_price_1 = float(main_info[13][i]) / float(coal_price[1][j])
            for j in range(len(coal_price[0])):
                sku_list.append(str(coal_price[0][j]))
                base_price = base_price_1 * float(coal_price[1][j])
                if (base_price <= 50):
                    base_price = base_price // 1 - 0.01
                else:
                    left_num = base_price  // 10 * 10
                    right_num =  base_price * 10 // 10 - left_num
                    right_num_2 = right_num // 5 * 5 + 4
                    base_price = left_num + right_num_2 + 0.99
                    if ((abs((base_price - main_info[10][i]) / main_info[10][i]) <= 0.05 and
                        abs((base_price - main_info[10][i]) / main_info[10][i]) != 0) or
                        (main_info[17][i] / base_price < 1 / 1.05 and
                        main_info[17][i] != 0)):
                        right_num_2 = right_num // 5 * 5 - 1
                        base_price = left_num + right_num_2 + 0.99
                price_list.append(base_price)
    
    
        else:
            sku_list.append(str(main_info[7][i]))
            price_list.append(main_info[13][i])
            

        for cl in range(len(cluster_list)):
            for sku in range(len(sku_list)):
                zsd_prices[0].append(tk_so_cl[1][tk_so_cl[2].index(str(cluster_list[cl]))])
                zsd_prices[1].append("")
                zsd_prices[2].append(cluster_list[cl])
                zsd_prices[3].append("")
                zsd_prices[4].append(str(sku_list[sku]))
                zsd_prices[5].append(price_list[sku])
                zsd_prices[6].append("")
                zsd_prices[7].append("")
                zsd_prices[8].append(str(reaction_date))


zsd_prices = pd.DataFrame(zsd_prices)
zsd_prices = zsd_prices.transpose()
zsd_prices.to_excel(zsd_name,
                    index=False,
                    header=["СО", "Гр.Кластеров", "Кластер", "ТК", "Товар", "ПЦ единая по карте",
                            "МРЦ (табак)", "ЕИ (только табак)", "Дата начала цены"])     


#файл для новых апц

new_promo_prices = []

for i in range(19):
    new_promo_prices.append([])

n = 1
for i in range(goods_number):
    if (main_info[16][i] == "Заводим апц до 13.07"):
        cluster_list = []
        sku_list = []
        price_list = []
            
        if main_info[75][i] == "единая цена в городе":
            for j in range(len(tk_so_cl[0])):
                if str(tk_so_cl[5][j]) == str(main_info[6][i][:main_info[6][i].find("_ГМ_")]):
                    cluster_list.append(str(tk_so_cl[2][j]))
                cluster_list = list(set(cluster_list))     
        else:
            cluster_list.append(str(tk_so_cl[2][tk_so_cl[0].index(str(main_info[1][i]))]))
            
        if main_info[71][i] != 0:
            
            
            for j in range(len(pf[0])):
                if str(main_info[71][i]) == pf[1][j]:
                    sku_list.append(str(pf[0][j]))
                sku_list = list(set(sku_list))
            for j in range(len(sku_list)):
                price_list.append(main_info[13][i])
        elif main_info[73][i] == "расценка угля":
            for j in range(len(coal_price[0])):
                if main_info[7][i] == coal_price[0][j]:
                    base_price_1 = float(main_info[13][i]) / float(coal_price[1][j])
            for j in range(len(coal_price[0])):
                sku_list.append(str(coal_price[0][j]))
                base_price = base_price_1 * float(coal_price[1][j])
                if (base_price <= 50):
                    base_price = base_price // 1 - 0.01
                else:
                    left_num = base_price  // 10 * 10
                    right_num =  base_price * 10 // 10 - left_num
                    right_num_2 = right_num // 5 * 5 + 4
                    base_price = left_num + right_num_2 + 0.99
                    if ((abs((base_price - main_info[10][i]) / main_info[10][i]) <= 0.05 and
                        abs((base_price - main_info[10][i]) / main_info[10][i]) != 0) or
                        (main_info[17][i] / base_price < 1 / 1.05 and
                        main_info[17][i] != 0)):
                        right_num_2 = right_num // 5 * 5 - 1
                        base_price = left_num + right_num_2 + 0.99
                price_list.append(base_price)
    
    
        else:
            sku_list.append(str(main_info[7][i]))
            price_list.append(main_info[13][i])  


#             for j in range(len(pf[0])):
#                 if str(main_info[71][i]) == pf[1][j]:
#                     sku_list.append(str(pf[0][j]))
#                 sku_list = list(set(sku_list))
#         else:
#             sku_list.append(str(main_info[7][i]))
            
        
        for cl in range(len(cluster_list)):
            for sku in range(len(sku_list)):
                new_promo_prices[0].append(str(n))
                new_promo_prices[1].append(str(sku_list[sku]))
                if len(sku_list) == 1:
                    new_promo_prices[2].append(str(main_info[8][i]))
                elif main_info[73][i] == "расценка угля":
                    new_bc_prices[2].append(coal_price[2][coal_price[0].index(str(sku_list[sku]))])
                else:
                    new_promo_prices[2].append(pf[2][pf[0].index(str(sku_list[sku]))])
                new_promo_prices[3].append(price_list[sku])
                new_promo_prices[4].append(str(reaction_date))
                new_promo_prices[5].append("13.07.2023")
                new_promo_prices[6].append("")
                new_promo_prices[7].append("")
                new_promo_prices[8].append("")
                new_promo_prices[9].append("")
                new_promo_prices[10].append("")
                new_promo_prices[11].append("")
                new_promo_prices[12].append("")
                new_promo_prices[13].append("")
                new_promo_prices[14].append("")
                new_promo_prices[15].append(cluster_list[cl])
                new_promo_prices[16].append("")
                new_promo_prices[17].append("нет")
                new_promo_prices[18].append("M001")
                n += 1
                

new_promo_prices = pd.DataFrame(new_promo_prices)
new_promo_prices = new_promo_prices.transpose()
new_promo_prices.to_excel(new_promo_name,
                    index=False,
                    header=["№", "САП номер", "Наименование", "АПЦ", "Дата действия акции с", "Дата действия акции По",
                            "ТК (ч/з запятую, не более 255 символов)", "ТК искл  (ч/з запятую, не более 255 символов)",
                            "Город  (ч/з запятую)", "ЗО (ч/з запятую)", "Все ТК (ГМ+СМ+ММ)", "ТК ГМ", "Формат ТК",
                            "ЗО (ТОЛЬКО для городов c 2-мя и 3-мя ЗО)", "ЕИЗ", "Ценовой кластер",
                            "Примечание (ограничение на карту или чек, например)",
                            "Требует пополнения ('TOP-1000' либо 'KVI'. По умолчанию без пополнения)", "Тема РМ"])

#файл для новых бу

n = 1
new_bc_prices = []

for i in range(19):
    new_bc_prices.append([])

new_date = datetime.strptime(reaction_date, '%d.%m.%Y')
new_date = new_date + timedelta(days=14)
new_date = str(new_date.strftime('%d.%m.%Y'))

for i in range(goods_number):
    if (main_info[16][i] == "Заводим бу"):
        cluster_list = []
        sku_list = []
        price_list = []
            

            
            
        if main_info[75][i] == "единая цена в городе":
            for j in range(len(tk_so_cl[0])):
                if str(tk_so_cl[5][j]) == str(main_info[6][i][:main_info[6][i].find("_ГМ_")]):
                    cluster_list.append(str(tk_so_cl[2][j]))
                cluster_list = list(set(cluster_list))     
        else:
            cluster_list.append(str(tk_so_cl[2][tk_so_cl[0].index(str(main_info[1][i]))]))
            
        if main_info[71][i] != 0:
            
            
            for j in range(len(pf[0])):
                if str(main_info[71][i]) == pf[1][j]:
                    sku_list.append(str(pf[0][j]))
                sku_list = list(set(sku_list))
            for j in range(len(sku_list)):
                price_list.append(main_info[13][i])
        elif main_info[73][i] == "расценка угля":
            for j in range(len(coal_price[0])):
                if main_info[7][i] == coal_price[0][j]:
                    base_price_1 = float(main_info[13][i]) / float(coal_price[1][j])
            for j in range(len(coal_price[0])):
                sku_list.append(str(coal_price[0][j]))
                base_price = base_price_1 * float(coal_price[1][j])
                if (base_price <= 50):
                    base_price = base_price // 1 - 0.01
                else:
                    left_num = base_price  // 10 * 10
                    right_num =  base_price * 10 // 10 - left_num
                    right_num_2 = right_num // 5 * 5 + 4
                    base_price = left_num + right_num_2 + 0.99
                    if ((abs((base_price - main_info[10][i]) / main_info[10][i]) <= 0.05 and
                        abs((base_price - main_info[10][i]) / main_info[10][i]) != 0) or
                        (main_info[17][i] / base_price < 1 / 1.05 and
                        main_info[17][i] != 0)):
                        right_num_2 = right_num // 5 * 5 - 1
                        base_price = left_num + right_num_2 + 0.99
                price_list.append(base_price)
    
    
        else:
            sku_list.append(str(main_info[7][i]))
            price_list.append(main_info[13][i]) 
            
            
            
            
#             for j in range(len(pf[0])):
#                 if str(main_info[71][i]) == pf[1][j]:
#                     sku_list.append(str(pf[0][j]))
#                 sku_list = list(set(sku_list))
#         else:
#             sku_list.append(str(main_info[7][i]))
            
        for cl in range(len(cluster_list)):
            for sku in range(len(sku_list)):
                new_bc_prices[0].append(str(n))
                new_bc_prices[1].append(str(sku_list[sku]))
                if len(sku_list) == 1:
                    new_bc_prices[2].append(str(main_info[8][i]))
                elif main_info[73][i] == "расценка угля":
                    new_bc_prices[2].append(coal_price[2][coal_price[0].index(str(sku_list[sku]))])
                else:
                    new_bc_prices[2].append(pf[2][pf[0].index(str(sku_list[sku]))])
                new_bc_prices[3].append(price_list[sku])
                new_bc_prices[4].append(str(reaction_date))
                new_bc_prices[5].append(str(new_date))
                new_bc_prices[6].append("")
                new_bc_prices[7].append("")
                new_bc_prices[8].append("")
                new_bc_prices[9].append("")
                new_bc_prices[10].append("")
                new_bc_prices[11].append("")
                new_bc_prices[12].append("")
                new_bc_prices[13].append("")
                new_bc_prices[14].append("")
                new_bc_prices[15].append(cluster_list[cl])
                if ((str(main_info[4][i]) == "P12" or str(main_info[4][i]) == "P38") and 
                    str(main_info[74][i]) != "TOP_P1"):
                    new_bc_prices[16].append("24")
                else:
                    new_bc_prices[16].append("3")
                new_bc_prices[17].append("нет")
                new_bc_prices[18].append("M001")
                n += 1
                

new_bc_prices = pd.DataFrame(new_bc_prices)
new_bc_prices = new_bc_prices.transpose()
new_bc_prices.to_excel(new_bc_name,
                    index=False,
                    header=["№", "САП номер", "Наименование", "АПЦ", "Дата действия акции с", "Дата действия акции По",
                            "ТК (ч/з запятую, не более 255 символов)", "ТК искл  (ч/з запятую, не более 255 символов)",
                            "Город  (ч/з запятую)", "ЗО (ч/з запятую)", "Все ТК (ГМ+СМ+ММ)", "ТК ГМ", "Формат ТК",
                            "ЗО (ТОЛЬКО для городов c 2-мя и 3-мя ЗО)", "ЕИЗ", "Ценовой кластер",
                            "Примечание (ограничение на карту или чек, например)",
                            "Требует пополнения ('TOP-1000' либо 'KVI'. По умолчанию без пополнения)", "Тема РМ"])


#файл для смены апц

n = 1
p36_p43_cities = ["Санкт-Петербург", "Москва", "Новосибирск", "Краснодар", "Екатеринбург"]


promo_prices_change = []
for i in range(9):
    promo_prices_change.append([])

    
for i in range(goods_number):
    if (main_info[16][i] == "Меняем апц"):
        tk_list = []
        sku_list = []
        price_list = []

        if (main_info[75][i] == "все СО"):
            tk_list = list(set(tk_so_cl[0]))

        elif main_info[75][i] == "в 1 СО":
            for j in range(len(tk_so_cl[0])):
                if str(main_info[0][i]) == str(tk_so_cl[1][j]):
                    tk_list.append(str(tk_so_cl[0][j]))
                tk_list = list(set(tk_list))
        elif main_info[75][i] == "основные города":
            for j in range(len(tk_so_cl[0])):
                if str(tk_so_cl[5][j]) in p36_p43_cities:
                    tk_list.append(str(tk_so_cl[0][j]))
                tk_list = list(set(tk_list)) 
                
        elif main_info[75][i] == "единая цена в городе":
            for j in range(len(tk_so_cl[0])):
                if str(tk_so_cl[5][j]) == str(main_info[6][i][:main_info[6][i].find("_ГМ_")]):
                    tk_list.append(str(tk_so_cl[0][j]))
                tk_list = list(set(tk_list))    
        else:
            for j in range(len(tk_so_cl[0])):
                if str(tk_so_cl[6][j]) == str(main_info[6][i]):
                    tk_list.append(str(tk_so_cl[0][j]))
                tk_list = list(set(tk_list))
            
        if main_info[71][i] != 0:
            
            
            for j in range(len(pf[0])):
                if str(main_info[71][i]) == pf[1][j]:
                    sku_list.append(str(pf[0][j]))
                sku_list = list(set(sku_list))
            for j in range(len(sku_list)):
                price_list.append(main_info[13][i])
        elif main_info[73][i] == "расценка угля":
            for j in range(len(coal_price[0])):
                if main_info[7][i] == coal_price[0][j]:
                    base_price_1 = float(main_info[13][i]) / float(coal_price[1][j])
            for j in range(len(coal_price[0])):
                sku_list.append(str(coal_price[0][j]))
                base_price = base_price_1 * float(coal_price[1][j])
                if (base_price <= 50):
                    base_price = base_price // 1 - 0.01
                else:
                    left_num = base_price  // 10 * 10
                    right_num =  base_price * 10 // 10 - left_num
                    right_num_2 = right_num // 5 * 5 + 4
                    base_price = left_num + right_num_2 + 0.99
                    if ((abs((base_price - main_info[10][i]) / main_info[10][i]) <= 0.05 and
                        abs((base_price - main_info[10][i]) / main_info[10][i]) != 0) or
                        (main_info[17][i] / base_price < 1 / 1.05 and
                        main_info[17][i] != 0)):
                        right_num_2 = right_num // 5 * 5 - 1
                        base_price = left_num + right_num_2 + 0.99
                price_list.append(base_price)
        else:
            sku_list.append(str(main_info[7][i]))
            price_list.append(main_info[13][i])
            
          
            
#             for j in range(len(pf[0])):
#                 if str(main_info[71][i]) == pf[1][j]:
#                     sku_list.append(str(pf[0][j]))
#                 sku_list = list(set(sku_list))
#         else:
#             sku_list.append(str(main_info[7][i]))
    
        for tk in range(len(tk_list)):
            for sku in range(len(sku_list)):
                promo_prices_change[0].append(str(n))
                if len(sku_list) == 1:
                    promo_prices_change[1].append(str(main_info[8][i]))
                elif main_info[73][i] == "расценка угля":
                    new_bc_prices[2].append(coal_price[2][coal_price[0].index(str(sku_list[sku]))])
                else:
                    promo_prices_change[1].append(pf[2][pf[0].index(str(sku_list[sku]))])
                promo_prices_change[2].append(str(sku_list[sku]))
                promo_prices_change[3].append(str(tk_list[tk]))
                promo_prices_change[4].append("")
                promo_prices_change[5].append("")
                promo_prices_change[6].append(price_list[sku])
                promo_prices_change[7].append(main_info[67][i])
                promo_prices_change[8].append(str(reaction_date))
                n += 1
                                        

        
promo_prices_change = pd.DataFrame(promo_prices_change)
promo_prices_change = promo_prices_change.transpose()
promo_prices_change.to_excel(change_promo_name,
                              index=False,
                              header=["№", "Наименование", "Товар", "ТК", "Все ТК", "Код города",
                                      "Цена", "Акция\БУ", "Дата действия"]) 
    
    


################################################
print("Выполнение программы завершено.")
print (str(time.time() - start))
print('Созданы файлы - ' + final_name + ', ' + hp_name +', ' + zsd_name + ', ' 
      + change_promo_name + ', ' + new_promo_name)


...Подготовка уникальных СО и товаров...
11.61373519897461
...Импорт основной информации...
46.30869483947754
...Импорт конкурентов...
79.4654049873352
...Поиск пц...
120.88401961326599
...Поиск зц...
128.91950631141663
Товар не найден в файле зц (-2):  1
...Поиск бу...
134.73099040985107
...Поиск цс...
134.76586508750916
...Импорт для скрытого промо...
179.84774923324585
...Расчет скрытого промо...
216.7232527732849
...Доработка файла скрытого промо...
222.7650773525238
...Экспорт скрытого промо...
222.77403283119202
...Расчет цен...
223.4552400112152
...Экспорт в эксель...
233.10803771018982
Выполнение программы завершено.
295.50077080726624
Созданы файлы - final_26_2.xlsx, hp_file_to_sap_26_2.xlsx, zsd_prices_26_2.xlsx, promo_prices_change_26_2.xlsx, promo_prices_new_26_2.xlsx


In [18]:
#     if ((str(main_info[18][i]) not in pilot_1_markets) and
#         main_info[73][i] == "пилот 1"):
#         main_info[73][i] = 0
print(str(main_info[18][86]))
print(str(main_info[18][86])  not in pilot_1_markets )
print(str(main_info[18][1]))
print(str(main_info[18][1])  not in pilot_1_markets )

АШАН
True
ПЯТЕРОЧКА
False
